# Exploratory Data Using Youtube API

Building a model to get and request data from Youtube API in order to create dataset for the project.

In [1]:
# Import nessesary libraries
import pandas as pd
# Google API
# %pip install google-api-python-client # Use this command to directly install the package from Jupyter Notebook
from googleapiclient.discovery import build

In [2]:
# Define the API key
api_key = '...' # Enter your API key here
youtube = build('youtube', 'v3', developerKey=api_key)
#Define Keywords
keywords = ['tự học lập trình', 'Vpop', 'chương trình giải trí', 'Vlog ăn uống', 'Vlog học','thể thao','trending','lịch sử']

# 1. Get nessesary data from Youtube API

In [14]:
#Search videos to get Channel ID
channel_ID_list = []
def get_channel_id(keyword):
    next_page_token = None
    total_results = 0
    while total_results <= 200:
        request = youtube.search().list(
            part='snippet',
            q=keyword,
            type='video',
            maxResults=50,
            order='relevance',
            regionCode='VN',
            pageToken=next_page_token
        )
        response = request.execute()
        for item in response['items']:
            channel_ID = item['snippet']['channelId']
            if channel_ID not in channel_ID_list:
                channel_ID_list.append(channel_ID)                
        total_results += len(response['items'])
        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
for keyword in keywords:
    get_channel_id(keyword)

In [20]:
#Get data for channel_dataset
channel_dataset = []
channel_playlistID = []
def get_channel_data(channel_ID):
    request = youtube.channels().list(
        part='snippet,statistics,contentDetails',
        id=channel_ID
    )
    response = request.execute()
    
    for item in response['items']:
        if int(item['statistics']['subscriberCount']) >= 300:
            channel_data = {
                'channel_ID': item['id'],
                'channel_title': item['snippet']['title'],
                'channel_description': item['snippet']['description'],
                'channel_subscriberCount': item['statistics']['subscriberCount'],
                'channel_viewCount': item['statistics']['viewCount'],
                'channel_videoCount': item['statistics']['videoCount'],
            }
            playlist_id = item['contentDetails']['relatedPlaylists']['uploads']
            channel_dataset.append(channel_data)
            channel_playlistID.append(playlist_id)
for channel_ID in channel_ID_list:
    get_channel_data(channel_ID)


In [21]:
df = pd.DataFrame(channel_dataset)
df.info()
numeric_cols = ['channel_viewCount', 'channel_subscriberCount', 'channel_videoCount']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors = 'coerce' )
df.info()
df.to_csv('channel_dataset.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770 entries, 0 to 769
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   channel_ID               770 non-null    object
 1   channel_title            770 non-null    object
 2   channel_description      770 non-null    object
 3   channel_subscriberCount  770 non-null    object
 4   channel_viewCount        770 non-null    object
 5   channel_videoCount       770 non-null    object
dtypes: object(6)
memory usage: 36.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770 entries, 0 to 769
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   channel_ID               770 non-null    object
 1   channel_title            770 non-null    object
 2   channel_description      770 non-null    object
 3   channel_subscriberCount  770 non-null    int64 
 4   chan

In [27]:
video_dataset = []

def get_video_data(playlist_ID):
    request = youtube.playlistItems().list(
        part='snippet,contentDetails',
        playlistId=playlist_ID,
        maxResults=10,
    )
    response = request.execute()
    
    video_ids = [item['contentDetails']['videoId'] for item in response['items']]
    if video_ids:
        video_request = youtube.videos().list(
            part='snippet,statistics,contentDetails',
            id=','.join(video_ids),
        )
        video_response = video_request.execute()
        
        for item in video_response['items']:
            video_data = {
                'channel_ID': item['snippet']['channelId'],
                'channel_title': item['snippet']['channelTitle'],
                'video_ID': item['id'],
                'video_title': item['snippet']['title'],                
                'video_description': item['snippet']['description'],
                'video_tags': item['snippet'].get('tags', None),
                "video_category": item.get('brandingSettings', {}).get('video', {}).get('keywords', ''),
                'video_viewCount': item['statistics']['viewCount'],
                'video_likeCount': item['statistics'].get('likeCount', 0),
                'video_commentCount': item['statistics'].get('commentCount', 0),
                'video_dateModified': item['snippet']['publishedAt'],
                'video_duration': item['contentDetails'].get('duration', None)
            }
            video_dataset.append(video_data)

for playlist_id in channel_playlistID:
    get_video_data(playlist_id)

In [ ]:
df = pd.read_csv('video_dataset.csv')

# Hàm lấy thumbnail từ YouTube API
def get_video_thumbnail(video_id):
    try:
        # Gọi API để lấy thông tin video
        request = youtube.videos().list(
            part="snippet",
            id=video_id
        )
        response = request.execute()
        
        # Kiểm tra nếu video có tồn tại và lấy thumbnail
        if "items" in response and len(response["items"]) > 0:
            thumbnails = response["items"][0]["snippet"]["thumbnails"]
            # Lấy thumbnail có độ phân giải cao nhất (maxres nếu có)
            if 'maxres' in thumbnails:
                return thumbnails['maxres']['url']
            elif 'high' in thumbnails:
                return thumbnails['high']['url']
            elif 'medium' in thumbnails:
                return thumbnails['medium']['url']
            elif 'default' in thumbnails:
                return thumbnails['default']['url']
        return None
    except Exception as e:
        print(f"Error fetching thumbnail for {video_id}: {e}")
        return None

# Hàm bổ sung thumbnail vào DataFrame
def add_thumbnails_to_dataset(df):
    # Tạo một cột mới 'video_thumbnail' để lưu thumbnail
    df['video_thumbnail'] = df['video_ID'].apply(get_video_thumbnail)
    return df

df_with_thumbnails = add_thumbnails_to_dataset(df)

df_with_thumbnails.to_csv('video_dataset.csv', index=False)

print("Thumbnails added successfully!")

Thumbnails added successfully!


In [45]:
import isodate

df = pd.DataFrame(video_dataset)
df.info()
numeric_cols = ['video_viewCount', 'video_likeCount', 'video_commentCount']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors = 'coerce' )
df['video_dateModified'] = pd.to_datetime(df['video_dateModified'], errors='coerce')
df['video_duration'] = df['video_duration'].apply(lambda x: f"{int(isodate.parse_duration(x).total_seconds() // 3600)}:{int((isodate.parse_duration(x).total_seconds() % 3600) // 60):02d}:{int(isodate.parse_duration(x).total_seconds() % 60):02d}" if pd.notnull(x) else None)
df.info()
df.to_csv('video_dataset.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7628 entries, 0 to 7627
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   channel_ID          7628 non-null   object
 1   channel_title       7628 non-null   object
 2   video_ID            7628 non-null   object
 3   video_title         7628 non-null   object
 4   video_description   7628 non-null   object
 5   video_tags          4295 non-null   object
 6   video_category      7628 non-null   object
 7   video_viewCount     7628 non-null   object
 8   video_likeCount     7628 non-null   object
 9   video_commentCount  7628 non-null   object
 10  video_dateModified  7628 non-null   object
 11  video_duration      7603 non-null   object
dtypes: object(12)
memory usage: 715.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7628 entries, 0 to 7627
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype              
---  -----

In [ ]:
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Bước 1: Xử lý giá trị null trong cột 'tags'
# Thay thế các giá trị null bằng chuỗi rỗng
df['video_tags'] = df['video_tags'].fillna('')

# Bước 2: Kết hợp 'title' và 'tags' để tạo thành một cột mới cho phân tích
df['text'] = df['video_title'] + ' ' + df['video_tags'].apply(lambda tags: ' '.join(tags) if isinstance(tags, list) else tags)

# Bước 3: Tiền xử lý văn bản
def preprocess_text(text):
    text = text.lower()  # Chuyển văn bản thành chữ thường
    text = ''.join([char for char in text if char not in string.punctuation])  # Loại bỏ dấu câu
    text = ' '.join([word for word in text.split() if word not in ENGLISH_STOP_WORDS])  # Loại bỏ stopwords
    return text

# Áp dụng tiền xử lý cho cột 'text'
df['processed_text'] = df['text'].apply(preprocess_text)

# Bước 4: Áp dụng TfidfVectorizer để tính toán TF-IDF
vectorizer = TfidfVectorizer(
    max_df=0.85,             # Loại bỏ từ xuất hiện trong quá nhiều tài liệu
    min_df=5,                # Loại bỏ từ chỉ xuất hiện trong ít tài liệu
    max_features=5000        # Giới hạn số lượng từ khóa
)

# Tính toán ma trận TF-IDF
tfidf_matrix = vectorizer.fit_transform(df['processed_text'])

# Bước 5: Lấy tên các từ khóa từ TF-IDF
tfidf_feature_names = vectorizer.get_feature_names_out()

# Bước 6: Tạo DataFrame từ ma trận TF-IDF
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_feature_names)

# Bước 8: Hiển thị các từ khóa có trọng số cao nhất
top_keywords = tfidf_df.mean(axis=0).sort_values(ascending=False).head(50)
print("Top 50 keywords:")
print(top_keywords)


Top 50 keywords:
shorts        0.030722
học           0.024522
trending      0.023875
vlog          0.021364
tin           0.016807
dance         0.015098
việt          0.013931
tiktok        0.013535
study         0.012738
anh           0.012494
funny         0.012002
nam           0.011979
trình         0.011344
ăn            0.010852
2024          0.010727
của           0.010142
lập           0.010006
nhất          0.009779
nhạc          0.009729
ngày          0.009415
phim          0.009174
sử            0.009066
video         0.008634
làm           0.008563
mình          0.008435
lịch          0.008422
sinh          0.008289
viral         0.008094
bóng          0.008083
tập           0.007731
song          0.007655
quốc          0.007585
mới           0.007582
viralvideo    0.007362
người         0.007327
đi            0.007273
tức           0.007153
những         0.007131
đá            0.007100
có            0.006970
kpop          0.006896
và            0.006858
sự            0.0

In [ ]:
import emoji

def remove_emojis(text):
    return emoji.replace_emoji(text, replace='')

def assign_category(row):
    # Chuyển tất cả các giá trị trong video_tags và video_title thành chữ thường và loại bỏ emoji
    tags = ' '.join(row['video_tags']).lower() if isinstance(row['video_tags'], list) else row['video_tags'].lower()
    tags = remove_emojis(tags)
    title = remove_emojis(row['video_title'].lower())
    
    # Kiểm tra từ khóa trong cột video_tags hoặc video_title
    if any(keyword in tags or keyword in title for keyword in ['học', 'sử', 'study', 'du', 'lập', 'trình', 'code']):
        return 'Education'
    
    elif any(keyword in tags or keyword in title for keyword in ['trending', 'vlog', 'tiktok', 'short', 'funny', 'viral', '2024', 'viral video']):
        return 'Trending'
    
    elif any(keyword in tags or keyword in title for keyword in ['dance', 'funny', 'phim', 'kpop', 'comedy', 'nhạc']):
        return 'Entertainment'
    
    elif any(keyword in tags or keyword in title for keyword in ['tập', 'bóng']):
        return 'Fitness'
    
    elif 'ăn' in tags or 'ăn' in title:
        return 'Food'
    
    return 'Others'
# Áp dụng hàm vào DataFrame để tạo cột 'category'
df['category'] = df.apply(assign_category, axis=1)
video_dataset = df.to_dict('records')
df.to_csv('video_dataset.csv', index=False)

In [ ]:
video_df = pd.DataFrame(video_dataset)
# Drop the specified columns
video_df = video_df.drop(columns=['video_category', 'text', 'processed_text'])
video_dataset = video_df.to_dict('records')
video_df.to_csv('video_dataset.csv', index=False)

In [120]:
video_df['like_view_ratio'] = video_df['video_likeCount'] / video_df['video_viewCount']
video_df['comment_view_ratio'] = video_df['video_commentCount'] / video_df['video_viewCount']
video_df.info()
video_df.to_csv('video_dataset.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7628 entries, 0 to 7627
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   channel_ID          7628 non-null   object             
 1   channel_title       7628 non-null   object             
 2   video_ID            7628 non-null   object             
 3   video_title         7628 non-null   object             
 4   video_description   7628 non-null   object             
 5   video_tags          7628 non-null   object             
 6   video_viewCount     7628 non-null   int64              
 7   video_likeCount     7628 non-null   int64              
 8   video_commentCount  7628 non-null   int64              
 9   video_dateModified  7628 non-null   datetime64[ns, UTC]
 10  video_duration      7603 non-null   object             
 11  category            7628 non-null   object             
 12  like_view_ratio     7591 non-null 

In [ ]:
video_df = pd.DataFrame(video_dataset)
video_df = video_df[video_df['video_commentCount'] >= 10]
video_dataset = video_df.to_dict('records')


In [ ]:
from googleapiclient.errors import HttpError
comment_dataset = []
def get_comment_data(video_ID):
    request = youtube.commentThreads().list(
        part='snippet',
        videoId=video_ID,
        maxResults=10,
        textFormat='plainText',
    )
    
    try:
        response = request.execute()
        for item in response.get('items', []):
            comment_data = {
                'video_ID': video_ID,
                'comment_ID': item['id'],
                'comment_author': item['snippet']['topLevelComment']['snippet'].get('authorDisplayName', 'Unknown'),
                'comment_text': item['snippet']['topLevelComment']['snippet']['textDisplay'],
                'comment_date': item['snippet']['topLevelComment']['snippet']['publishedAt'],
                'comment_likeCount': item['snippet']['topLevelComment']['snippet']['likeCount'],
            }
            comment_dataset.append(comment_data)

    except HttpError as e:
        print(f"Error fetching comments for video {video_ID}: {e}")

for video in video_dataset:
    get_comment_data(video['video_ID'])

In [126]:
comment_df = pd.DataFrame(comment_dataset)
comment_df.info()
comment_df['comment_date'] = pd.to_datetime(comment_df['comment_date'], errors='coerce')
comment_df.info()
comment_df.to_csv('comment_dataset.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35038 entries, 0 to 35037
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   video_ID           35038 non-null  object
 1   comment_ID         35038 non-null  object
 2   comment_author     35038 non-null  object
 3   comment_text       35038 non-null  object
 4   comment_date       35038 non-null  object
 5   comment_likeCount  35038 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 1.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35038 entries, 0 to 35037
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   video_ID           35038 non-null  object             
 1   comment_ID         35038 non-null  object             
 2   comment_author     35038 non-null  object             
 3   comment_text       35038 non-null  object            

In [ ]:
from textblob import TextBlob
def get_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity
comment_data = pd.read_csv('comment_dataset.csv')
comment_data['comment_text'] = comment_data['comment_text'].fillna('')
comment_data['sentiment_score'] = comment_data['comment_text'].apply(get_sentiment)
comment_data.to_csv('comment_dataset.csv', index=False)